# Problem 2 - Neural Network Training and Backpropagation

Lets import and inspect our data as shown in the example code:

In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
%matplotlib inline

data = loadmat('ex3data1.mat')
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [37]:
X = data['X']
y = data['y']

X.shape, y.shape

((5000, 400), (5000, 1))

In [38]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
y_onehot = encoder.fit_transform(y)
y_onehot.shape

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(5000, 10)

In [39]:
y[0], y_onehot[0,:]

(array([10], dtype=uint8), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]))

## 2.1

In [40]:
# sigmoid()
def sigmoid(z):
    return 1 / (1 + np.exp(-2*z))

Our sigmoid function is defined with -2z instead of z as we are using the scaled sigmoid.

## 2.2

In [41]:
# forward_propagate()
def forward_propagate(X, theta1, theta2, theta3):
    m = X.shape[0]

    #input layer
    a1 = np.insert(X, 0, values=np.ones(m), axis=1)
    z2 = a1 * theta1.T

    #1st hidden layer
    a2 = np.insert(sigmoid(z2), 0, values=np.ones(m), axis=1)
    z3 = a2 * theta2.T

    #2nd hidden layer
    a3 = np.insert(sigmoid(z3), 0, values=np.ones(m), axis=1)
    z4 = a3 * theta3.T

    h = sigmoid(z4)

    return a1, z2, a2, z3, a3, z4, h

In the forward_propagate function we added a3 and z4 terms to account for out 2nd hidden layer.

## 2.3

In [42]:
# cost() without regularization
def cost_without_regularization(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

    # reshape the parameter array into parameter matrices for each layer
    end_theta1 = hidden_size * (input_size + 1)
    end_theta2 = end_theta1 + hidden_size * (hidden_size + 1)

    theta1 = np.matrix(np.reshape(params[:end_theta1], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[end_theta1:end_theta2], (hidden_size, (hidden_size + 1))))
    theta3 = np.matrix(np.reshape(params[end_theta2:], (num_labels, (hidden_size + 1))))

    # run the feed-forward pass
    a1, z2, a2, z3, a3, z4, h = forward_propagate(X, theta1, theta2, theta3)

    # compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)

    J = J / m

    return J


In the cost_without_regularization function we added theta 3 to account for our 2nd hidden layer.

Lets create sample parameters to test our functions as we define them.

In [43]:
# initial setup
input_size = 400
hidden_size = 20
num_labels = 10
learning_rate = 1

# randomly initialize a parameter array of the size of the full network's parameters
params = (np.random.random(size=hidden_size * (input_size + 1) +
                           hidden_size * (hidden_size + 1) +
                           num_labels * (hidden_size + 1)) - 0.5) * 0.25

m = X.shape[0]
X = np.matrix(X)
y = np.matrix(y)

# unravel the parameter array into parameter matrices for each layer
end_theta1 = hidden_size * (input_size + 1)
end_theta2 = end_theta1 + hidden_size * (hidden_size + 1)

theta1 = np.matrix(np.reshape(params[:end_theta1], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(params[end_theta1:end_theta2], (hidden_size, (hidden_size + 1))))
theta3 = np.matrix(np.reshape(params[end_theta2:], (num_labels, (hidden_size + 1))))

print(theta1.shape, theta2.shape, theta3.shape)


(20, 401) (20, 21) (10, 21)


In [44]:
a1, z2, a2, z3, a3, z4, h = forward_propagate(X, theta1, theta2, theta3)
a1.shape, z2.shape, a2.shape, z3.shape, a3.shape, z4.shape, h.shape

((5000, 401),
 (5000, 20),
 (5000, 21),
 (5000, 20),
 (5000, 21),
 (5000, 10),
 (5000, 10))

In [45]:
cost_without_regularization(params, input_size, hidden_size, num_labels, X, y_onehot, learning_rate)

7.100031060813976

In [46]:
# cost() with regularization
def cost_with_regularization(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

  # reshape the parameter array into parameter matrices for each layer
    end_theta1 = hidden_size * (input_size + 1)
    end_theta2 = end_theta1 + hidden_size * (hidden_size + 1)

    theta1 = np.matrix(np.reshape(params[:end_theta1], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[end_theta1:end_theta2], (hidden_size, (hidden_size + 1))))
    theta3 = np.matrix(np.reshape(params[end_theta2:], (num_labels, (hidden_size + 1))))

    # run the feed-forward pass
    a1, z2, a2, z3, a3, z4, h = forward_propagate(X, theta1, theta2, theta3)

    # compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)

    J = J / m

    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) +
                                             np.sum(np.power(theta2[:,1:], 2)) +
                                             np.sum(np.power(theta3[:,1:], 2)))

    return J

In the cost_with_regularization function in addition to theta 3 variable we also altered our cost regulariztion term J to account for the theta 3 (our second hidden layer).

In [47]:
cost_with_regularization(params, input_size, hidden_size, num_labels, X, y_onehot, learning_rate)

7.104578168789641

We can see that cost increases with regularization, but this increase is very small (0.004)

## 2.4

In [48]:
# sigmoid_gradient()
def sigmoid_gradient(z):
    return 2 * np.multiply(sigmoid(z), (1 - sigmoid(z)))

In sigmoid_gradient funciton we multipied our sigmoid gradient with 2 wo account for our scaled sigmoid.

## 2.5

In [49]:
# backprop() without regularization
def backprop_without_regularization(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

    # reshape the parameter array into parameter matrices for each layer
    end_theta1 = hidden_size * (input_size + 1)
    end_theta2 = end_theta1 + hidden_size * (hidden_size + 1)

    theta1 = np.matrix(np.reshape(params[:end_theta1], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[end_theta1:end_theta2], (hidden_size, (hidden_size + 1))))
    theta3 = np.matrix(np.reshape(params[end_theta2:], (num_labels, (hidden_size + 1))))

    # run the feed-forward pass
    a1, z2, a2, z3, a3, z4, h = forward_propagate(X, theta1, theta2, theta3)

    # initializations
    J = 0
    delta1 = np.zeros(theta1.shape)
    delta2 = np.zeros(theta2.shape)
    delta3 = np.zeros(theta3.shape)

    # compute the cost
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)

    J = J / m

    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) +
                                             np.sum(np.power(theta2[:,1:], 2)) +
                                             np.sum(np.power(theta3[:,1:], 2)))
    # perform backpropagation
    for t in range(m):
        a1t = a1[t,:]
        z2t = z2[t,:]
        a2t = a2[t,:]
        z3t = z3[t,:]
        a3t = a3[t,:]

        ht = h[t,:]
        yt = y[t,:]

        d4t = ht - yt

        z3t = np.insert(z3t, 0, values=np.ones(1))
        d3t = np.multiply((theta3.T * d4t.T).T, sigmoid_gradient(z3t))

        z2t = np.insert(z2t, 0, values=np.ones(1))
        d2t = np.multiply((theta2.T * d3t[:,1:].T).T, sigmoid_gradient(z2t))

        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + (d3t[:,1:]).T * a2t
        delta3 = delta3 + d4t.T * a3t

    delta1 = delta1 / m
    delta2 = delta2 / m
    delta3 = delta3 / m

    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2), np.ravel(delta3)))

    return J, grad

In the backprop_without_regularization fuction we needed to alter the backpropagation step by adding z3t and a3t terms as well as d4t and delta 3 to account for our additional 2nd hidden layer. We also added our 2nd hidden layer (delta 3) to the "grad" array with unraveled gradient matrices.

In [50]:
J, grad = backprop_without_regularization(params, input_size, hidden_size, num_labels, X, y_onehot, learning_rate)
J, grad.shape

(7.104578168789641, (8650,))

In [51]:
# backprop() with regularization
def backprop_with_regularization(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

    # reshape the parameter array into parameter matrices for each layer
    end_theta1 = hidden_size * (input_size + 1)
    end_theta2 = end_theta1 + hidden_size * (hidden_size + 1)

    theta1 = np.matrix(np.reshape(params[:end_theta1], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[end_theta1:end_theta2], (hidden_size, (hidden_size + 1))))
    theta3 = np.matrix(np.reshape(params[end_theta2:], (num_labels, (hidden_size + 1))))

    # run the feed-forward pass
    a1, z2, a2, z3, a3, z4, h = forward_propagate(X, theta1, theta2, theta3)

    # initializations
    J = 0
    delta1 = np.zeros(theta1.shape)  # (25, 401)
    delta2 = np.zeros(theta2.shape)  # (10, 26)
    delta3 = np.zeros(theta3.shape)  # (10, 27)

    # compute the cost
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)

    J = J / m

    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) +
                                             np.sum(np.power(theta2[:,1:], 2)) +
                                             np.sum(np.power(theta3[:,1:], 2)))
    # perform backpropagation
    for t in range(m):
        a1t = a1[t,:]
        z2t = z2[t,:]
        a2t = a2[t,:]
        z3t = z3[t,:]
        a3t = a3[t,:]

        ht = h[t,:]
        yt = y[t,:]

        d4t = ht - yt

        z3t = np.insert(z3t, 0, values=np.ones(1))
        d3t = np.multiply((theta3.T * d4t.T).T, sigmoid_gradient(z3t))

        z2t = np.insert(z2t, 0, values=np.ones(1))
        d2t = np.multiply((theta2.T * d3t[:,1:].T).T, sigmoid_gradient(z2t))

        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + (d3t[:,1:]).T * a2t
        delta3 = delta3 + d4t.T * a3t

    delta1 = delta1 / m
    delta2 = delta2 / m
    delta3 = delta3 / m

    # add the gradient regularization term
    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * learning_rate) / m
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:] * learning_rate) / m
    delta3[:,1:] = delta3[:,1:] + (theta3[:,1:] * learning_rate) / m

    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2), np.ravel(delta3)))

    return J, grad

In the backprop_with_regularization in addition to the delta 1 and delta 2 regularization terms we also added the delta 3 regularization term.

In [52]:
J, grad = backprop_with_regularization(params, input_size, hidden_size, num_labels, X, y_onehot, learning_rate)
J, grad.shape

(7.104578168789641, (8650,))

## 2.6

Lets train our 3-layered neural network:

In [53]:
from scipy.optimize import minimize

# minimize the objective function
fmin = minimize(fun=backprop_with_regularization, x0=params, args=(input_size, hidden_size, num_labels, X, y_onehot, learning_rate),
                method='TNC', jac=True, options={'maxiter': 250})
fmin

<ipython-input-53-4310fc965557>:5: OptimizeWarning: Unknown solver options: maxiter
  fmin = minimize(fun=backprop_with_regularization, x0=params, args=(input_size, hidden_size, num_labels, X, y_onehot, learning_rate),


 message: Converged (|f_n-f_(n-1)| ~= 0)
 success: True
  status: 1
     fun: 0.09706570345205856
       x: [ 1.068e+00 -2.191e-07 ...  1.112e+00 -5.328e-01]
     nit: 61
     jac: [ 1.284e-05 -4.383e-11 ... -1.189e-05 -5.827e-06]
    nfev: 1163

In [54]:
X = np.matrix(X)
# reshape the parameter array into parameter matrices for each layer
end_theta1 = hidden_size * (input_size + 1)
end_theta2 = end_theta1 + hidden_size * (hidden_size + 1)

theta1 = np.matrix(np.reshape(fmin.x[:end_theta1], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(fmin.x[end_theta1:end_theta2], (hidden_size, (hidden_size + 1))))
theta3 = np.matrix(np.reshape(fmin.x[end_theta2:], (num_labels, (hidden_size + 1))))

a1, z2, a2, z3, a3, z4, h = forward_propagate(X, theta1, theta2, theta3)

## 2.7

Lets make foward predictions with our model and compute accuracy:

In [55]:
y_pred = np.array(np.argmax(h, axis=1) + 1)
y_pred

array([[10],
       [10],
       [10],
       ...,
       [ 9],
       [ 9],
       [ 9]])

In [56]:
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y)]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print('accuracy = {0}%'.format(accuracy * 100))

accuracy = 100.0%


## 2.8

**Answer:**
Looking at our model we can notice that it achieved accuracy of 100% this can mean two things - a) we built an exceptioinal model that captures more informaiton about our data with the 2nd hidden layer or b) our model is overfitting and thus will not generalize well on the new data. Comparing it to the original model with 1 hidden layer (instead of 2) we can notice that the amout the accuracy approved is minimal - from 99.22% to 100%. Solely looking at this we should consider if adding the additional layer is worth it as it leads to very little improvment (since the original model is doing very well) but it does take longer to train (requires more computational resources which can get very expensive very quickly) and it also potentially leads to overfitting. Due to this I do not think that adding the addiotional hidden layer is valuable as it makes the model more complex which could (and maybe did) lead to overfitting and is more computationally expensive which is not worth it as the original model preformed very well.